In [35]:
import requests
import time
import json
from tqdm import tqdm
from pprint import pprint
 
class VkUser:
    url = 'https://api.vk.com/method/'

    def __init__(self, token, version):
        self.token = token
        self.version = version
        self.params = {
            'access_token': self.token,
            'v': self.version,
        }
        self.owner_id = requests.get(self.url + 'users.get', self.params).json()['response'][0]['id']

    def get_photos_links(self, user_id: str, qty_photos: str):
        if user_id is None:
            user_id = self.owner_id
        followers_url = self.url + 'photos.get'
        followers_params = {
            'album_id': 'profile',
            'count': qty_photos,
            'user_id': user_id,
            'extended': 1
        }
        res = requests.get(followers_url, params={**self.params, **followers_params})
        complete_list = res.json()['response']['items']
        photos_to_upload = []
        url_names = []
        for line in complete_list:
            photo_info = {}
            qty_likes = line['likes']['count']
            photo_date = line['date']
            biggest_photo = line['sizes'][-1]
            photo_name = f'{qty_likes}.jpg'
            for item in photos_to_upload:
                if item['file_name'] == photo_name:
                    photo_name = f'{qty_likes}_{photo_date}.jpg'
            url_names.append([biggest_photo['url'], photo_name])
            photo_info['file_name'] = photo_name
            photo_info['size'] = biggest_photo['type']
            photos_to_upload.append(photo_info)
        with open('about_photos.json', 'w') as json_file:
            json.dump(photos_to_upload, json_file)
            pprint(photos_to_upload)
        return url_names
        

class YaUploader:
    def __init__(self, token: str, user_id: str):
        self.token = token
        self.user_id = user_id

    def create_folder(self):
        url = 'https://cloud-api.yandex.net/v1/disk/resources'
        params = {'path': self.user_id}
        headers = {'Authorization': self.token}
        response = requests.put(url, params=params, headers=headers)
        if response.status_code == 201:
            print(f'Папка "{self.user_id}" создана на диске.')
        else:
            error_message = response.json()['message']
            pprint(f'Ошибка код - {response.status_code}: {error_message}')

    def upload_from_url(self, lst_link_name):
        for link, name in tqdm(lst_link_name):
            response = requests.post('https://cloud-api.yandex.net/v1/disk/resources/upload',
                                     params={'path': f"{self.user_id}/{name}",
                                             'url': link},
                                     headers={'Authorization': f'OAuth {self.token}'})
            if response.status_code == 202:
                time.sleep(0.1)
        return


if __name__ == '__main__':
    user_id = ''
    qty_photos = str(input('Введите количество фото для загрузки: '))
    ya_token = ''
    token_vk = ''
    vk_client = VkUser(token_vk, '5.130')
    list_of_links = vk_client.get_photos_links(user_id, qty_photos)
    uploader = YaUploader(ya_token, user_id)
    uploader.create_folder()
    uploader.upload_from_url(list_of_links)

[{'file_name': '266.jpg', 'size': 'z'},
 {'file_name': '346.jpg', 'size': 'y'},
 {'file_name': '465.jpg', 'size': 'z'},
 {'file_name': '476.jpg', 'size': 'z'},
 {'file_name': '572.jpg', 'size': 'y'},
 {'file_name': '469.jpg', 'size': 'z'},
 {'file_name': '294.jpg', 'size': 'y'},
 {'file_name': '321.jpg', 'size': 'y'}]
Папка "57827859" создана на диске.


100%|██████████| 8/8 [00:04<00:00,  1.85it/s]
